In [287]:
import re
import pandas as pd
from collections import Counter, defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize

In [801]:
def rule_based_parse_BCJ(path):
    '''Given file path (text file) of negligence cases, finds static 
    information within the case (information that can be pattern matched)
    Expects a B.C.J. case format (British Columbia Judgments)
    The following fields are currently implemented:
    - Case Title
    - Judge Name
    - Registry
    - Year
    - Decision Length (in paragraphs)
    The following fields are still being implemented & tweaked:
    - Multiple Defendants
    Arguments: doc (String): The case in text format following the form used in the DOCX to TXT notebook
    Returns: case_parsed_data (list) of case_dict (Dictionary): List of Dictionaries with rule based parsable fields filled in
    '''
    with open(path, encoding='utf-8') as document:
        document_data = document.read()
        
    document_data = document_data.split('End of Document\n') # Always split on 'End of Document\n'
    case_parsed_data = []
    for i in range(len(document_data)):
        case_dict = dict() 
        case = document_data[i]
        case = case.strip() # Make sure to strip!
        if len(case) == 0: # Skip empty lines
            continue
        
        lines = case.split('\n')
        if len(lines) < 2:
            print(case)
        case_title = lines[0]
        case_type = lines[1]

        if 'R. v.' in case_title: # Skip crown cases
            continue

        if 'British Columbia Judgments' in case_type: # Make sure we're dealing with a B.C.J. case
            
            # Fields that can be found via pattern matching
            if re.search('contributory negligence', case, re.IGNORECASE):
                contributory_negligence_raised = True
            else:
                contributory_negligence_raised = False
            case_number = re.search(r'\/P([0-9]+)\.txt', path).group(1)
            decision_len = re.search(r'\(([0-9]+) paras\.?\)', case) # e.g.) (100 paras.)
            registry = re.search(r'(Registry|Registries): ?([A-Za-z0-9 ]+)', case) # e.g.) Registry: Vancouver
            if registry:
                registry = registry.group(2).strip()
            else:
                registry = re.search(r'([A-Za-z ]+) Registry No.', case) # Alt form e.g.) Vancouver Registory No. XXX
                if registry:
                    registry = registry.group(1).strip()
                else:
                    registry = re.search(r'([A-Za-z ]+) No. S[0-9]*', case)
                    if registry:
                        registry = registry.group(1).strip()
                    else:
                        print('WARNING: Registry could not be found (This shouldn\'t occur!)')
            # Fields that are always in the same place
            judge_name = lines[4].strip()
            case_title = lines[0].strip()
            # Extract year from case_title (in case we want to make visualizations, etc.)
            year = re.search(r'20[0-2][0-9]', case_title) # Limit regex to be from 2000 to 2029
            if year:
                year = year.group(0)
            else:
                # Rare case: Sometimes the title is too long. Rely on Heard date.
                year = re.search(r'Heard:.* ([2][0][0-2][0-9])', case)
                if year:
                    year = year.group(1)
                else:
                    print('WARNING: Year not found')
            case_dict['case_number'] = '%s of %s'%(i+1+((int(case_number)-1)*50), case_number)
            case_dict['case_title'] = case_title
            case_dict['year'] = year
            case_dict['registry'] = registry
            case_dict['judge'] = judge_name
            case_dict['decision_length'] = decision_len.group(1)
            case_dict['multiple_defendants'] = rule_based_multiple_defendants_parse(case)
            case_dict['contributory_negligence_raised'] = contributory_negligence_raised
        if case_dict != dict():
            case_parsed_data.append(case_dict)
    return case_parsed_data

In [799]:
def rule_based_multiple_defendants_parse(doc):
    ''' Work in progress. Subject to minor changes to Regex patterns.
    
    TODO:
        - Clarify solicitor/client cases with Lachlan
        - Clarify cases that say "IN MATTER OF ...", currently returning 'UNK' for these
    
    -----
    
    Given a case. Uses regex/pattern-matching to determine whether we have multiple defendants.
    For the most part the logic relies on whether the langauge used implies plurality or not.
    
    Arguments: doc (String): The case in text format following the form used in the DOCX to TXT notebook
    Returns: response (String, 'Y', 'N', or 'UNK')
    
    '''
    
    # Cases with (Re) in the title always have one person involved
    # May drop these cases depending on advice from Lachlan.
    if '(Re)' in doc.split('\n')[0]:
        return 'N'
    
    # Case 1)
    # Traditional/most common. Of form "Between A, B, C, Plaintiff(s), X, Y, Z Defendant(s)"
    # Can successfully cover ~98% of data
    regex_between_plaintiff_claimant = re.search(r'(Between.*([P|p]laintiff[s]?|[C|c]laimant[s]?|[A|a]ppellant[s]?|[P|p]etitioner[s]?).*([D|d]efendant[s]?|[R|r]espondent[s]?).*\n)', doc)
    
    # Match found
    if regex_between_plaintiff_claimant:
        text = regex_between_plaintiff_claimant.group(0).lower()
        if 'defendants' in text or 'respondents' in text:
            return 'Y'
        elif 'defendant' in text or 'respondent' in text:
            return 'N'
    
    # If not found, try other less common cases
    else:
        # Case 2)
        # Sometimes it does not mention the name of the second item. (Defendent/Respondent)
        # We can estimate if there are multiple based on the number of "," in the line (Covers all cases in initial data)
        regex_missing_defendent = re.search(r'(Between.*([P|p]laintiff[s]?|[C|c]laimant[s]?|[A|a]ppellant[s]?|[P|p]etitioner[s]?).*\n)', doc)
        if regex_missing_defendent:
            text = regex_missing_defendent.group(0).lower()
            if len(text.split(',')) > 5:
                return 'Y'
            else:
                return 'N'
            
        else:
            
            # Case 3A) solicitor-client
            # Some cases have a solicitor (lawyer) and a client
            # Currently assuming the second item is the defendant
            regex_solicitor_client = re.search(r'(Between.*([S|s]olicitor[s]?).*([C|c]lient[s]?))', doc)
            if regex_solicitor_client:
                text = regex_solicitor_client.group(0).lower()
                if 'clients' in text:
                    return 'Y'
                else:
                    return 'N'
            else:
                # Case 3B) client - solicitor
                regex_client_solicitor = re.search(r'(Between.*([C|c]lient[s]?).*([S|s]olicitor[s]?))', doc)
                if regex_client_solicitor:
                    text = regex_client_solicitor.group(0).lower()
                    if 'solicitors' in text:
                        return 'Y'
                    else:
                        return 'N'
                else:
                    return 'UNK'
        
        

In [803]:
docs_feats = rule_based_parse_BCJ('./data/Lexis Cases txt/P1.txt')
docs_feats

[{'case_number': '1 of 1',
  'case_title': 'Mawani v. Pitcairn, [2012] B.C.J. No. 1819',
  'year': '2012',
  'registry': 'Vancouver',
  'judge': 'S.F. Kelleher J.',
  'decision_length': '115',
  'multiple_defendants': 'Y',
  'contributory_negligence_raised': True},
 {'case_number': '2 of 1',
  'case_title': 'Ediger (Guardian ad litem of) v. Johnston, [2009] B.C.J. No. 564',
  'year': '2009',
  'registry': 'Vancouver',
  'judge': 'H.J. Holmes J.',
  'decision_length': '350',
  'multiple_defendants': 'Y',
  'contributory_negligence_raised': False},
 {'case_number': '3 of 1',
  'case_title': 'Furness v. Guest, [2010] B.C.J. No. 1388',
  'year': '2010',
  'registry': 'Nanaimo',
  'judge': 'D.A. Halfyard J.',
  'decision_length': '97',
  'multiple_defendants': 'Y',
  'contributory_negligence_raised': True},
 {'case_number': '4 of 1',
  'case_title': 'Howell v. Machi, [2017] B.C.J. No. 2016',
  'year': '2017',
  'registry': 'Vancouver',
  'judge': 'H. MacNaughton',
  'decision_length': '525'

In [83]:
seq = re.compile(r"conclusions?:?.*", re.MULTILINE|re.IGNORECASE)
for match in seq.finditer(docs[1]):
    print(match.group(0))

CONCLUSIONS
Conclusion re Future Care
CONCLUSION - DAMAGE AWARD
CONCLUSIONS
conclusions are these.
conclusion of the forceps attempt.
conclusion of the forceps attempt.
conclusion that it is more likely than not the conduct of the defendant was a cause in fact of the result. A mere possibility of such causation is not enough; and when the matter remains one of pure speculation or conjecture, or the probabilities are at best evenly balanced, it becomes the duty of the court to direct a verdict for the defendant.
conclusion in Cherry v. Borsman, as described above, may no longer apply. This enlarged position effectively subsumes Dr. Johnston's original position. However, because of the manner in which the case was presented and argued in the trial, with further written submissions after judgment was reserved, it seems most convenient and responsive to the weight and direction of counsels' submissions to address first the matter of informed consent and the fetus, and then the larger quest

### Trying to find patterns in damages awarded

Print all paragraphs with  dollar values in them 

In [379]:
no_paras = re.search(r'\(([0-9|,]+) paras?\.?\)', docs[0]).group(1)
pattern = r'[\W|\w]?(?=\n[0-9]{1,%s}[\xa0|\s| ]{2})'%len(no_paras)

paras_split = re.split(pattern, docs[0])
paras_split[-1]

'\n115\xa0\xa0In the result, I find liability is apportioned 50% to each party.\nS.F. KELLEHER J.\n'

In [866]:
def paragraph_tokenize(file_name):
    '''Print document by paragraph number. If quantity mentioned, print values.
    Return list of lists of numbered paragraphs per document'''

    with open(file_name) as file:
        docs = file.read().split('End of Document\n')[:-1]
    
    docs_split = []
    contributory_negligence = []
    no_paras_per_doc = []
#     for doc in docs:
    for doc in docs:
        if re.search('contributory negligence', doc, re.IGNORECASE):
            contributory_negligence.append(True)
        else:
            contributory_negligence.append(False)
        doc_data = []
        print('@@NEW DOC@@')
        lines = doc.split('\n')
        if not 'British Columbia Judgments' in lines[1]:
            continue
        doc_data.append(lines[0])
        no_paras = re.search(r'\(([0-9|,]+) paras?\.?\)', doc).group(1)
        no_paras_per_doc.append(no_paras)
        print('NUM OF PARAS: %s'%no_paras)
#         if no_paras:
#             no_paras = no_paras.group(1)
        pattern = r'[\W|\w]?(?=\n[0-9]{1,%s}[\xa0]{2})'%len(no_paras)

        paras_split = re.split(pattern, doc)
        paras = []
        for para in paras_split:
#             prev_para_no = 0
            
            para_start = re.match(r'^\n([0-9]{1,%s})[\xa0]{2}'%len(no_paras), para)
            if para_start:
#                 if int(para_start.group(1)) == int(prev_para_no)+1 or prev_para_no == 0:
                print(para)
                paras.append(para)
#                 quantity_mentioned = re.findall(r'(?:\$?(?<![\n|A-Za-z])(?:[0-9]{1,3},?\.?[0-9][0-9]?[0-9]?)%?(?![\xa0]{2}|:)+)', para)
                percent_mentioned = re.findall(r'(?:[1-9][0-9]*%)', para)
                if len(percent_mentioned) > 0:
                    print('PERCENT MENTIONED')
                    print(percent_mentioned)
                if para in paras_split[-5:]:
                    print('ONE OF LAST 5 PARAGRAPHS - DAMAGES HERE???')
                print('======')
        doc_data.extend(paras)
        docs_split.append(doc_data)
    return docs_split, contributory_negligence, no_paras_per_doc

In [1007]:
path = './data/Lexis Cases txt/P2.txt'
doc_split, contr_neg, no_paras = paragraph_tokenize(path)

@@NEW DOC@@
@@NEW DOC@@
NUM OF PARAS: 166

1   The plaintiff, Ms. Nelson, claims damages from the defendants based on negligence. The Plaintiff asserts that the defendant, Ms. Tucker, while performing her duties as a nurse employed by the defendant B.C. Women's Hospital, breached a duty of care owed to Ms. Nelson, and that injury resulted. There is no issue that the defendants owed a duty to Ms. Nelson; the question is whether it was breached, and, if it were, did injury ensue. The breach is alleged to have occurred during the period of labour leading to the birth of Ms. Nelson's daughter in 2009. Specifically, Ms. Nelson claims that while she was under the influence of an epidural anaesthetic, and unable to control her legs, Ms. Tucker allowed her left leg to drop, which caused injury to Ms. Nelson's left hip

2  Ms. Nelson was 39 years old, married to Dr. Beaudoin, a chiropractor, and pregnant with her first child, when she went into labour on April 30, 2009. She went to B.C. Women's

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [991]:
doc_split[0][-1]

"\n166\xa0\xa0If needed, the parties may speak to costs.\nR. JOHNSTON J.\n* * * * *\nCORRIGENDUM\nReleased: November 2, 2015\n[1] The Reasons for Judgment dated October 26, 2015, 2015 BCSC 1941, have been corrected as follows:\n[2] On the first page, W. Pickett, Articled Student, has been added as counsel for the plaintiff.\n[3] The last sentence in paragraph 1 is corrected as follows:\nSpecifically, Ms. Nelson claims that while she was under the influence of an epidural anaesthetic, and unable to control her legs, Ms. Tucker allowed her left leg to drop, which caused injury to Ms. Nelson's left hip.\nR. JOHNSTON J.\n"

Try to pick out quantities mentioned per paragraph and the context around each

In [804]:
doc_feats = defaultdict(lambda: defaultdict(dict))


for i in range(len(doc_split)):
    doc_name = doc_split[i][0]
    doc = doc_split[i]
    for j in range(1, len(doc)):
        para = doc[j]
        tokens = para.split()
        quantity_mentioned = re.findall(r'(?:\$?(?<![\n|A-Za-z])(?:[0-9]{1,3},?\.?[0-9][0-9]?[0-9]?)%?(?![\xa0]{2}|:)+)', para)
        if len(quantity_mentioned) > 0:
            doc_feats[doc_name]['paragraph %s'%(j+1)]['quantities_found']  = True
            for quantity in quantity_mentioned:
#                 loc = [i for i, token in enumerate(tokens) if quantity in token]
#                 print(loc)
                loc = para.find(quantity)
                if loc - 25 >=0 and loc+25 < len(para):
                    doc_feats[doc_name]['paragraph %s'%(j+1)][quantity] = para[loc-25:loc+25+1]
                elif loc - 25 < 0 and loc+25 < len(para):
                    beg = abs(loc -25)
                    doc_feats[doc_name]['paragraph %s'%(j+1)][quantity] = para[loc-25+beg:loc+25+1]
                elif loc - 25 > 0 and loc+25 > len(para): 
                    beg = abs(loc -25)
                    end = loc+25
                    doc_feats[doc_name]['paragraph %s'%(j+1)][quantity] = para[loc-25+beg:loc+25-end+1]
                
        else:
            doc_feats[doc_name]['paragraph %s'%(j+1)]['quantities_found']  = False
#         doc_feats[doc_name]['paragraph %s'%(j+1)]['quantities'] = quantity_mentioned
    break

In [992]:
doc_feats['Mawani v. Pitcairn, [2012] B.C.J. No. 1819']

defaultdict(dict,
            {'2 of 115': {'score': 0.017391304347826087},
             '107 of 115': {'90%': 'travel. Burnyeat J. apportioned liability 90% to the plaintiff and 10%',
              '10%': '90% to the plaintiff and 10% to the defendant',
              'score': 0.9304347826086956},
             '115 of 115': {'50%': 'I find liability is apportioned 50% to each party. S.F. KELLEHER',
              'score': 1.0},
             'contributory_negligence': True})

### % Reduction due to contributory negligence 

In [1033]:
def paragraph_tokenize(path):
    ''' Takes a file name as input and returns a list of lists of numbered paragraphs per document in file,
    where the first item in each sublist is the document name.
    ---------
    Input: path (str) - path where .txt file of cases is located
    Return: docs_split(list) - list of lists of numbrered paragraphs per document'''

    with open(path) as file:
        docs = file.read().split('End of Document\n')[:-1]
    
    docs_split = []
    contributory_negligence = []
    paras_per_doc = []
    
    for doc in docs:
        if re.search('contributory negligence', doc, re.IGNORECASE):
            contributory_negligence.append(True)
        else:
            contributory_negligence.append(False)
        doc_data = []
        lines = doc.split('\n')
        if not 'British Columbia Judgments' in lines[1]:
            continue
        doc_data.append(lines[0])
        decision_length = re.search(r'\(([0-9|,]+) paras?\.?\)', doc).group(1)
        paras_per_doc.append(decision_length)
        # split paragraphs on newline, paragraph number, two spaces
        pattern = r'[\W|\w]?(?=\n[0-9]{1,%s}[\xa0]{2})'%len(decision_length)
        paras_split = re.split(pattern, doc)
        
        paras = []
        for para in paras_split:   
            # make sure the paragraph starts with the correct characters
            para_start = re.match(r'^\n([0-9]{1,%s})[\xa0]{2}'%len(decision_length), para)
            if para_start:
                paras.append(para)
        doc_data.extend(paras)
        docs_split.append(doc_data)
    return docs_split, contributory_negligence, paras_per_doc

In [1022]:
import os
path = "./data/Lexis Cases txt/"
list_of_files = os.listdir(path)


In [1068]:
dfs = []
documents_with_contr_neg = []
documents_with_contr_neg_no_percent = []

for file in list_of_files:
    print(file)
    if file != ".DS_Store":
        docs_split, contributory_negligence, paras_per_doc = paragraph_tokenize(path + file)
        doc_feats = defaultdict(lambda: defaultdict(dict))
        count = 0
        for i in range(len(doc_split)):
            doc_name = doc_split[i][0]
            doc = doc_split[i]
            contr_neg = contributory_negligence[i]
            num_paras = paras_per_doc[i]
            percent_in_last_para = []
            for j in range(1, len(doc)):
                para = doc[j]

                tokens = para.split()
                percent_mentioned = re.findall(r'(?:[1-9][0-9]*%|percent)', para, re.IGNORECASE) # percent pattern
                if len(percent_mentioned) > 0:
                    for percent in percent_mentioned:
                        loc = [i for i, token in enumerate(tokens) if percent in token] 
                        if len(loc)> 0:
                            if len(loc) > 1:
                                count += 1
                            loc = loc[0]

                            if loc - 5 >=0 and loc+5 < len(tokens):
                                doc_feats[doc_name]['%s of %s'%(j, num_paras)][percent] = " ".join(tokens[loc-5:loc+5+1])
                            elif loc - 5 < 0 and loc+5 < len(tokens):
                                beg = abs(loc -5)
                                doc_feats[doc_name]['%s of %s'%(j, num_paras)][percent] = " ".join(tokens[loc-5+beg:loc+5+1])
                            elif loc - 5 > 0 and loc+5 > len(tokens): 
                                doc_feats[doc_name]['%s of %s'%(j, num_paras)][percent] = " ".join(tokens[loc-5:len(tokens)])
                    doc_feats[doc_name]['%s of %s'%(j, num_paras)]['score'] = int(j)/int(num_paras)
            doc_feats[doc_name]['contributory_negligence'] = contr_neg
        for document in doc_feats:
            if doc_feats[document]['contributory_negligence']:
                if len(doc_feats[document]) > 1:
                    mod =  doc_feats[document].copy()
                    mod.pop('contributory_negligence')
                    df = pd.DataFrame(mod)

                    df = df.transpose().reset_index().rename(columns = {'index':"paragraph no."}).melt(value_name='context', var_name='value', id_vars=["paragraph no.", 'score'])
                    dfs.append(df.dropna())
                    documents_with_contr_neg.append(document)
                else:
                    print(document)
                    documents_with_contr_neg_no_percent.append(document)

P1.txt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
Jurisevic v. Rideout, [2013] B.C.J. No. 965
Waters v. Mariash, [2012] B.C.J. No. 1296
Antrobus v. Antrobus, [2014] B.C.J. No. 2532
Gu (Litigation guardian of) v. Friesen, [2013] B.C.J. No. 686
Knight v. Imperial Tobacco Canada Ltd., [2007] B.C.J. No. 1461
P21.txt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


IndexError: list index out of range

In [1003]:
dfs = []
documents_with_contr_neg = []
documents_with_contr_neg_no_percent = []

In [1009]:

for document in doc_feats:
    if doc_feats[document]['contributory_negligence']:
        if len(doc_feats[document]) > 1:
            mod =  doc_feats[document].copy()
            mod.pop('contributory_negligence')
            df = pd.DataFrame(mod)

            df = df.transpose().reset_index().rename(columns = {'index':"paragraph no."}).melt(value_name='context', var_name='value', id_vars=["paragraph no.", 'score'])
            dfs.append(df.dropna())
            documents_with_contr_neg.append(document)
        else:
            print(document)
            documents_with_contr_neg_no_percent.append(document)


Jurisevic v. Rideout, [2013] B.C.J. No. 965
Waters v. Mariash, [2012] B.C.J. No. 1296
LeBlanc v. Vancouver Island Health Authority, [2014] B.C.J. No. 844
Antrobus v. Antrobus, [2014] B.C.J. No. 2532
Bakken v. Goddard, [2010] B.C.J. No. 2075


Of the cases where contributory negligence was mentioned and no % or 'percent' found:
P1.txt:
* Tait v. Dumansky, [2012] B.C.J. No. 451 =  'no need to apportion cause' -> no contributory negligence
* McIvor v. MLK Pharmacies Ltd., [2016] B.C.J. No. 2515 = open case! 
* Mezo v. Malcolm, [2013] B.C.J. No. 2150 = no contributory negligence
P2.txt:
* Jurisevic v. Rideout, [2013] B.C.J. No. 965
* Waters v. Mariash, [2012] B.C.J. No. 1296
* LeBlanc v. Vancouver Island Health Authority, [2014] B.C.J. No. 844
* Antrobus v. Antrobus, [2014] B.C.J. No. 2532
* Bakken v. Goddard, [2010] B.C.J. No. 2075

In [953]:
df.transpose().reset_index().rename(columns = {'index':"paragraph no."}).melt(value_name='context', var_name='value', id_vars=["paragraph no.", 'score'])

,paragraph no.,score,value,context
0,2 of 115,0.0173913,90%,NaN
1,107 of 115,0.930435,90%,travel. Burnyeat J. apportioned liability 90% ...
2,115 of 115,1,90%,NaN
3,2 of 115,0.0173913,10%,NaN
4,107 of 115,0.930435,10%,90% to the plaintiff and 10% to the defendant
5,115 of 115,1,10%,NaN
6,2 of 115,0.0173913,50%,NaN
7,107 of 115,0.930435,50%,NaN
8,115 of 115,1,50%,I find liability is apportioned 50% to each pa...


In [1010]:
len(dfs)

51

In [1012]:
# this is for p1.txt and p2.txt
nested_df = pd.DataFrame({'Df of perecent mentions':dfs}, index= documents_with_contr_neg)
nested_df.iloc[-1,:]['Df of perecent mentions']

,paragraph no.,score,value,context
0,29 of 144,0.201389,5%,fixed rate of interest at 12.5% per annum
2,35 of 144,0.243056,5%,mortgage funding to Belciug at 12.5% per annum...
5,30 of 144,0.208333,9%,"broker fee of $6,300, representing 9% of the l..."
9,30 of 144,0.208333,90%,and that he normally gets 90% of the broker's ...
13,30 of 144,0.208333,10%,"of the broker's fee, with 10% going to TMG. In..."
19,140 of 144,0.972222,70%,"as against the defendant, Binpal 70%; as again..."
23,140 of 144,0.972222,30%,"as against the defendants Dhaliwal/TMG, 30%. F..."


## Parsers to extract Case Summary and HELD sections

In [1150]:
def extract_case_summary_and_HELD(path):
    with open(path) as file:
        cases = file.read().split('End of Document\n')[:-1]

    for case in cases:
        lines = case.split('\n')
        case_title = lines[0]
        summary = re.search(r'\([0-9]{1,3} paras\.\)\ncase summary\n((.*\n+?)+)(?=HELD|(Statutes, Regulations and Rules Cited:))', case, re.IGNORECASE)
        if summary:
            summary = summary.group(1)
            money_mentioned = re.findall(r'\$[0-9|,]+', summary)
                    
        else:
            summary ='NO SUMMARY'
            money_mentioned = None
        print(case_title)
        print(money_mentioned)
        print(summary)
        print('======')
 
        


In [1152]:
extract_case_summary_and_HELD('./data/Lexis Cases txt/P1.txt')

Mawani v. Pitcairn, [2012] B.C.J. No. 1819
[]
Civil litigation — Civil procedure — Discovery — Examination for discovery — Use of examination in court — Trials — Summary trials — Application by plaintiff to determine liability for motor vehicle accident allowed in part — Plaintiff struck by defendant's vehicle while walking along roadway — Neither party had clear recollection of accident and little other admissible evidence — Answers provided by defendant during discovery that were reasonably connected to questions read in by plaintiff ordered read in — Both parties contravened Motor Vehicle Act — Defendant negligent as drove too fast for conditions and too close to curb — Plaintiff contributorily negligent because he failed to take adequate precautions as pedestrian — Liability apportioned equally as per statutory presumption — Supreme Court Civil Rules, Rules 9-7(15) and 12-5(49).
Tort law — Negligence — Duty and standard of care — Duty of care — Standard of care — Causation — Contri